**Author: Guillermo Raya Garcia**<br>
**Universitat Autònoma de Barcelona**
# Real Vs Fake Job Postings: An analysis

In [1]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats

In [2]:
# Funcio per a llegir dades en format csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

# Carreguem dataset d'exemple
dataset = load_dataset('fake_job_postings.csv')
data = dataset.values
labels = dataset.columns.values

## 1. Observing the dataset

In [32]:
dataset.head()

job_id                                      title            location  \
0       1                           Marketing Intern    US, NY, New York   
1       2  Customer Service - Cloud Video Production      NZ, , Auckland   
2       3    Commissioning Machinery Assistant (CMA)       US, IA, Wever   
3       4          Account Executive - Washington DC  US, DC, Washington   
4       5                        Bill Review Manager  US, FL, Fort Worth   

  department salary_range                                    company_profile  \
0  Marketing          NaN  We're Food52, and we've created a groundbreaki...   
1    Success          NaN  90 Seconds, the worlds Cloud Video Production ...   
2        NaN          NaN  Valor Services provides Workforce Solutions th...   
3      Sales          NaN  Our passion for improving quality of life thro...   
4        NaN          NaN  SpotSource Solutions LLC is a Global Human Cap...   

                                         description  \
0  Food52, a fast-growing, James Beard Award-winn...   
1  Organised - Focused - Vibrant - Awesome!Do you...   
2  Our client, located in Houston, is actively se...   
3  THE COMPANY: ESRI – Environmental Systems Rese...   
4  JOB TITLE: Itemization Review ManagerLOCATION:...   

                                        requirements  \
0  Experience with content management systems a m...   
1  What we expect from you:Your key responsibilit...   
2  Implement pre-commissioning and commissioning ...   
3  EDUCATION: Bachelor’s or Master’s in GIS, busi...   
4  QUALIFICATIONS:RN license in the State of Texa...   

                                            benefits  telecommuting  \
0                                                NaN              0   
1  What you will get from usThrough being part of...              0   
2                                                NaN              0   
3  Our culture is anything but corporate—we have ...              0   
4                              Full Benefits Offered              0   

   has_company_logo  has_questions employment_type required_experience  \
0                 1              0           Other          Internship   
1                 1              0       Full-time      Not Applicable   
2                 1              0             NaN                 NaN   
3                 1              0       Full-time    Mid-Senior level   
4                 1              1       Full-time    Mid-Senior level   

  required_education                   industry              function  \
0                NaN                        NaN             Marketing   
1                NaN  Marketing and Advertising      Customer Service   
2                NaN                        NaN                   NaN   
3  Bachelor's Degree          Computer Software                 Sales   
4  Bachelor's Degree     Hospital & Health Care  Health Care Provider   

   fraudulent  
0           0  
1           0  
2           0  
3           0  
4           0

In [26]:
def chartMe(content,colnames,rownames,precision=9,pre="", extra=""):
    #Making the rows of the chart
    chart=[]
    for i,valuesRow in enumerate(content):
        newRow=[str(pre+" "+rownames[i]+" "+extra)]
        newRow.extend([valuesRow])
        chart.append(newRow)
        
    #Setting up the precision, as requested by function call
    pd.set_option('precision', precision)
    seriousChart=pd.DataFrame(chart, columns=colnames).style.hide_index()
    pd.reset_option('precision')
    
    return seriousChart

In [28]:
chartMe(content=data[1],colnames=["Attribute","Value"],rownames=labels)

This dataset has 18 columns:

| Attribute | Data type | Attribute description |
| :-- | :-: | :-- |
|'job_id'| int | Row number |
|'title'| str | Name of the job offered |
|'location'| str | Location on which the job takes place, comprised of Country, State and City|
|'department'| str | Department of the offered job |
|'salary_range'| float| Salary range of the offered job |
|'company_profile'| str | Brief overview of the company |
|'description'| str | Description of the offered job |
|'requirements'| str | Requirements for the applicants to the job |
|'benefits'|float| Benefits included in the offer |
|'telecommuting'|int| Availability |
|'has_company_logo'|int| Existence and availability of a company logo |
|'has_questions'|int| TO-DO |
|'employment_type'|str| Type of employment (e.g.:'Full-time','Part-time', 'Minijob',etc.) |
|'required_experience'|str| Required experience for applicants |
|'required_education'|float| Required education for applicants |
|'industry'|float| Industry of the job offered |
|'function'|str| Function of the job offer |
|'fraudulent'|int| Fraudulent: 1 if yes, otherwise 0 |

In [33]:
dataset.description

0        Food52, a fast-growing, James Beard Award-winn...
1        Organised - Focused - Vibrant - Awesome!Do you...
2        Our client, located in Houston, is actively se...
3        THE COMPANY: ESRI – Environmental Systems Rese...
4        JOB TITLE: Itemization Review ManagerLOCATION:...
                               ...                        
17875    Just in case this is the first time you’ve vis...
17876    The Payroll Accountant will focus primarily on...
17877    Experienced Project Cost Control Staff Enginee...
17878    Nemsia Studios is looking for an experienced v...
17879    Who are we?Vend is an award winning web based ...
Name: description, Length: 17880, dtype: object

Since many important fields in this dataset are comprised solely of text, we will need to process them in some way so that we can compare them. And for that, we'll use scikit's tools for text feature extraction (which is exactly what we want: building informative and non-redundant values from our raw data in order to facilitate further tasks).

Taking a look at __[the documentation](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)__, we choose to start by running a monogram vectorisation on the 'description' and 'company profile' attributes, to start.